In [2]:
from time import sleep
import cv2 as cv
import numpy as np
import pandas as pd
#from math import abs

from functions import fa , miyangin_gir , door_tarin , fasele , nazdik_tarin , nazdik_tarin_m
from functions import siyah_peyda_kon , stackImages , sotone_detect , changeres , rescaleFrame 
from functions import predict_to_index , contours2X , gooshe_X , warp_image

from sklearn.preprocessing import StandardScaler



from tensorflow import keras



2022-03-19 04:56:21.030757: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mohammadsaleh/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-19 04:56:21.030777: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
#load model.h5
model = keras.models.load_model('model_(2022, 3, 4, 55).h5')


2022-03-19 04:56:22.519952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-19 04:56:22.520507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mohammadsaleh/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-19 04:56:22.520596: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mohammadsaleh/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-19 04:56:22.520666: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; 

In [4]:

def dade_film2df(contours , gooshe_ha , markaz , df):
    dict = {}

    for i in range(115):
        stx = "x"+str(i)
        sty = "y"+str(i)
        dict[stx] = 0
        dict[sty] = 0
    for i in range(len(contours)):
        dict["x"+str(i)] , dict["y"+str(i)] = door_tarin(contours[i] , markaz)-markaz
    if len(gooshe_ha[0]) == 4:
        
        dict["ULx"] , dict["ULy"]= np.array((int(gooshe_ha[0][0][0]),int(gooshe_ha[0][0][1]))) - markaz

        dict["URx"] , dict["URy"]= np.array((int(gooshe_ha[0][1][0]),int(gooshe_ha[0][1][1]))) - markaz

        dict["DRx"] , dict["DRy"]= np.array((int(gooshe_ha[0][3][0]),int(gooshe_ha[0][3][1]))) - markaz

        dict["DLx"] , dict["DLy"]= np.array((int(gooshe_ha[0][2][0]),int(gooshe_ha[0][2][1]))) - markaz
    else:
        dict["ULx"] , dict["ULy"] = None , None
        dict["URx"] , dict["URy"] = None , None
        dict["DRx"] , dict["DRy"] = None , None
        dict["DLx"] , dict["DLy"] = None , None


    df = df.append(dict, ignore_index=True)
    return df


In [5]:
df = pd.DataFrame()
capture = cv.VideoCapture('17.mp4')
j = 0
ret = True
while ret:
    
    ret , frame = capture.read()
    
    j+=1
    if ret:
        frame2 , mask , contours= siyah_peyda_kon(frame)
        if contours!= False:
            markaz = miyangin_gir(contours)
            frame3 = frame2.copy()
            
            X = np.array([contours2X(contours , markaz)])
            
            gooshe_ha = np.float32(gooshe_X(X , model)+markaz)
            frame4 = warp_image(frame, gooshe_ha)
            cv.circle(frame3,(int(gooshe_ha[0][0][0]),int(gooshe_ha[0][0][1])),7,(30,200,205),-1)
            cv.circle(frame3,(int(gooshe_ha[0][1][0]),int(gooshe_ha[0][1][1])),7,(0,255,0),-1)
            cv.circle(frame3,(int(gooshe_ha[0][2][0]),int(gooshe_ha[0][2][1])),7,(255,0,0),-1)
            cv.circle(frame3,(int(gooshe_ha[0][3][0]),int(gooshe_ha[0][3][1])),7,(255,255,0),-1)
            cv.putText(frame3 , str(j), (300,100) , cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,95), 2)
            cv.putText(frame3 , str(len(df)), (500,100) , cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,95), 2)
            
            frame1 = rescaleFrame(frame3 , 0.5)
            cv.imshow('frame', rescaleFrame(frame2))
            
            cv.imshow('gooshe_ha_be_tartib', frame1)
            cv.imshow('boresh_dade_shode', rescaleFrame(frame4 , 0.35))
            
            if cv.waitKey(0) == ord('l'):
                df = dade_film2df(contours , gooshe_ha , markaz , df)
            elif cv.waitKey(0) == ord('q'):
                break
            

cv.destroyAllWindows()


In [7]:
df

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,x114,y114,ULx,ULy,URx,URy,DRx,DRy,DLx,DLy
0,476.0,567.0,476.0,536.0,476.0,517.0,-482.0,512.0,475.0,499.0,...,0.0,0.0,-452.0,-499.0,474.0,-477.0,476.0,536.0,-482.0,512.0
1,473.0,563.0,473.0,532.0,472.0,514.0,-479.0,509.0,472.0,495.0,...,0.0,0.0,-448.0,-495.0,469.0,-473.0,473.0,532.0,-479.0,509.0
2,469.0,560.0,470.0,528.0,469.0,510.0,-475.0,507.0,468.0,492.0,...,0.0,0.0,-445.0,-491.0,466.0,-471.0,470.0,528.0,-475.0,507.0
3,464.0,541.0,-462.0,523.0,464.0,510.0,-462.0,492.0,463.0,492.0,...,0.0,0.0,-432.0,-483.0,459.0,-466.0,464.0,510.0,-462.0,492.0
4,462.0,549.0,463.0,517.0,-460.0,498.0,461.0,500.0,-460.0,481.0,...,0.0,0.0,-426.0,-471.0,458.0,-456.0,463.0,517.0,-460.0,498.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,477.0,560.0,476.0,529.0,-475.0,513.0,476.0,510.0,-475.0,495.0,...,0.0,0.0,-454.0,-491.0,470.0,-482.0,476.0,529.0,-475.0,513.0
108,476.0,559.0,477.0,527.0,-474.0,512.0,476.0,509.0,-474.0,494.0,...,0.0,0.0,-454.0,-491.0,470.0,-482.0,477.0,527.0,-474.0,512.0
109,476.0,559.0,476.0,528.0,-475.0,513.0,475.0,510.0,-475.0,495.0,...,0.0,0.0,-454.0,-491.0,469.0,-482.0,476.0,528.0,-475.0,513.0
110,473.0,555.0,473.0,548.0,473.0,524.0,-480.0,509.0,473.0,505.0,...,0.0,0.0,-459.0,-496.0,466.0,-487.0,473.0,548.0,-480.0,509.0


In [8]:
data_f = pd.read_csv('data.csv')
data_f

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,x114,y114,ULx,ULy,URx,URy,DRx,DRy,DLx,DLy
0,447,517,-441,498,447,486,-441,468,447,468,...,0,0,-404,-445,432,-441,447,486,-441,468
1,446,518,-441,499,446,487,-441,468,446,469,...,0,0,-405,-444,431,-440,446,487,-441,468
2,443,515,-440,496,443,484,442,467,-440,465,...,0,0,-403,-444,428,-440,443,484,-440,465
3,445,516,-440,497,445,485,-440,466,445,467,...,0,0,-403,-445,431,-438,445,485,-440,466
4,446,517,-441,497,445,486,-441,466,446,468,...,0,0,-403,-446,431,-441,445,486,-441,466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,475,486,-460,485,475,468,-460,468,474,450,...,0,0,-410,-415,422,-428,475,486,-460,485
867,-462,488,475,487,-462,470,475,469,-462,452,...,0,0,-412,-416,422,-429,475,487,-462,488
868,477,488,-462,488,477,470,-462,470,476,452,...,0,0,-411,-416,423,-429,477,488,-462,488
869,477,488,-463,488,477,469,-463,470,476,452,...,0,0,-411,-418,424,-428,477,488,-463,488


In [ ]:
sleep(30)

In [9]:
data_f = pd.merge( df , data_f , how = 'outer' )
data_f.to_csv('data.csv', index=False)

